In [9]:
import pandas as pd
from sklearn.linear_model import Ridge, Lasso, LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [4]:
data_regression = pd.read_csv('../../Exlr_Assignments/Datasets/50_Startups.csv')
data_regression.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [5]:
y_reg = data_regression['Profit']
X_category_reg = data_regression.iloc[:,-2]
X_numeric_reg = data_regression.iloc[:,:-2]

## 1.Embded techniques for feature slection

In [7]:
def get_embedded_report(X_numeric, X_category, y, estimator):
    
    # First normalize our predictors
    X_numeric_scaled = StandardScaler().fit_transform(X_numeric)
    y_scaled     = StandardScaler().fit_transform(y.values.reshape(-1,1))
    # Save the columns name into variable named 'features'
    X_category_dummies = pd.get_dummies(X_category)
    features = list(X_numeric_reg.columns) + list(X_category_dummies.columns)
        # Fit the regressor
    X = np.concatenate([X_numeric_scaled, X_category_dummies.values], axis=1)

    model = estimator.fit(X, y_scaled)
    try:
        c = model.coef_
    except:
        c = model.feature_importances_
    finally:
        return c, features

### 1.1.Feature selection using Regularization lasso technique

In [10]:
# Using L1-regularization
coef, features = get_embedded_report(X_numeric_reg, X_category_reg, y_reg, Lasso(alpha=0.1))
list(zip(features,coef))

[('R&D Spend', 0.849256095682484),
 ('Administration', -0.0),
 ('Marketing Spend', 0.032693579958810816),
 ('California', -0.0),
 ('Florida', 0.0),
 ('New York', -0.0)]

##### Administration and state seems to be less significant

### 1.2.Feature selection using Regularization Ridge technique

In [11]:
# Using L2-regularization
coef, features = get_embedded_report(X_numeric_reg, X_category_reg, y_reg, Ridge())
list(zip(features,coef[0]))

[('R&D Spend', 0.8780288777147082),
 ('Administration', -0.008085007680925026),
 ('Marketing Spend', 0.10909503223068648),
 ('California', -0.0024425845607658667),
 ('Florida', 0.001444242340860628),
 ('New York', 0.00099834221990486)]

### 1.3.Feature selection using Regularization DecisionTreeRegressor technique

In [12]:
# Using feature_importances_'s DecisionTree
from sklearn.tree import DecisionTreeRegressor

coef, features = get_embedded_report(X_numeric_reg, X_category_reg, y_reg, DecisionTreeRegressor())
list(zip(features,coef))

[('R&D Spend', 0.9706854751075797),
 ('Administration', 0.00341762292952448),
 ('Marketing Spend', 0.019931677536507417),
 ('California', 0.0009344318230540478),
 ('Florida', 0.0046524953528572595),
 ('New York', 0.0003782972504769403)]

### in all these method the feature will less value can be dropped to improve the performance i.e Administration and State